# Team ID : NT2022TMID35712

# Project Name : A Gesture-based Tool for Sterile Browsing of Radiology Images

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 11.1 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.


In [5]:
!pip install tensorflow

     |████████████████████████████████| 1.3 MB 17.9 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [30]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HaJV5PfQ3pclB58vpjjNM39n-V1FojdGcHPeTGjkWTwF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-fu4hinhb0wo8v5'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket = bucket,Key = object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [31]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [32]:
# This library helps add support for large, multi-dimensional arrays and matrices
import numpy as np
#open source used for both ML and DL for computation
import tensorflow as tf
#it is a plain stack of layers
from tensorflow.keras.models import Sequential 
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten, Dropout
#Faltten-used fot flattening the input or change the dimension, MaxPooling2D-for downsampling the image for Convolutional layer
from tensorflow.keras.layers import Convolution2D,MaxPooling2D 
#Its used for different augmentation of the image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
#setting parameter for Image Data agumentation to the traing data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [34]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train',
                                            target_size=(64, 64),
                                            batch_size=3,
                                            color_mode='grayscale',
                                            class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test',
                                          target_size=(64, 64),
                                          batch_size=3,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [35]:
print(x_train.class_indices)#checking the number of classes

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [36]:
# Initializing the CNN
model = Sequential()

In [37]:
# First convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [38]:
# Second convolution layer and pooling
model.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [39]:
# Flattening the layers i.e. input layer
model.add(Flatten())

In [40]:
# Adding a fully connected layer, i.e. Hidden Layer
model.add(Dense(units=512 , activation='relu'))

In [41]:
# softmax for categorical analysis, Output Layer
model.add(Dense(units=6, activation='softmax')) 

In [42]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

In [43]:
# Compiling the CNN
# categorical_crossentropy for more than 2
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [44]:
# It will generate packets of train and test data for training
model.fit_generator(x_train,
                    steps_per_epoch = 594/3 , 
                    epochs = 25, 
                    validation_data = x_test,
                    validation_steps = 30/3 )

/tmp/wsuser/ipykernel_164/804983804.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


Epoch 1/25
198/198 [==============================] - 13s 64ms/step - loss: 1.2669 - accuracy: 0.4865 - val_loss: 1.1331 - val_accuracy: 0.4667
Epoch 2/25
198/198 [==============================] - 13s 64ms/step - loss: 0.5514 - accuracy: 0.7828 - val_loss: 0.5053 - val_accuracy: 0.8333
Epoch 3/25
198/198 [==============================] - 13s 64ms/step - loss: 0.3482 - accuracy: 0.8653 - val_loss: 0.4686 - val_accuracy: 0.9000
Epoch 4/25
198/198 [==============================] - 13s 63ms/step - loss: 0.2324 - accuracy: 0.9125 - val_loss: 0.3969 - val_accuracy: 0.9000
Epoch 5/25
198/198 [==============================] - 12s 63ms/step - loss: 0.1926 - accuracy: 0.9242 - val_loss: 0.3948 - val_accuracy: 0.8000
Epoch 6/25
198/198 [==============================] - 13s 63ms/step - loss: 0.1265 - accuracy: 0.9545 - val_loss: 0.2774 - val_accuracy: 0.8667
Epoch 7/25
198/198 [==============================] - 13s 65ms/step - loss: 0.1432 - accuracy: 0.9377 - val_loss: 0.2363 - val_accuracy:

In [45]:
# Save the model
model.save('gesture.h5')

In [46]:
!tar -zcvf Gesture-based-Radiology-Images.tgz gesture.h5

gesture.h5


In [47]:
ls 

Dataset/  Gesture-based-Radiology-Images.tgz  gesture.h5


In [48]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# IBM Deployment

In [49]:
!pip install ibm_watson_machine_learning

In [50]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://eu-de.ml.cloud.ibm.com",
    "apikey":"QDes4cGnjlTAKZIn2xuN2RJmG8LH1YQ1GQqghQz0Z89v"
}

client=APIClient(wml_credentials)

In [54]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [68]:
space_uid="ab1a8dc2-f593-4cc6-8882-5c36cff6c59b"
print("Space UID "+space_uid)

Space UID ab1a8dc2-f593-4cc6-8882-5c36cff6c59b


In [69]:
client.set.default_space(space_uid)

'SUCCESS'

In [70]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [71]:
import tensorflow as tf
tf.__version__

'2.7.2'

In [72]:
import keras as ks
ks.__version__

'2.7.0'

In [73]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9-horovod')

In [74]:
software_space_uid

'dda170cc-ca67-5da7-9b7a-cf84c6987fae'